### Reviews about Tada

In this notebook, we are going to perform topic modelling based on sentiments from various review sites. 

In [20]:
# Base
import os, re, string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from wordcloud import WordCloud

# NLTK
import nltk
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.metrics.distance import edit_distance
from spellchecker import SpellChecker
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem.wordnet import WordNetLemmatizer

# Topic Modelling
import pyLDAvis
import pyLDAvis.gensim
import gensim
from gensim import corpora
pyLDAvis.enable_notebook()

[nltk_data] Downloading package stopwords to D:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Data Preparation

We will first get the relevant files we need and thereafter extract them out into a list of reviews.

In [21]:
# Merge into 

path='.'

filename_sentiment_list=[]

for filename in os.listdir(path):
    if filename.endswith(".csv"):
        # Check what are the files in the folder
        print(os.path.join(path, filename))
        
        #Read the files and put to df
        df = pd.read_csv(filename,encoding = "ISO-8859-1")
        
        #Add it into a list
        try:
            filename_sentiment_list.extend(df['comment'].tolist())
            filename_sentiment_list.extend(df['tweet'].tolist())
            filename_sentiment_list.extend(df['title'].tolist())
        except:
            pass

print(filename_sentiment_list)

.\gplay_tada_en.csv
.\reddit_tada_singapore.csv
.\reddit_tada_singapore_extracted.csv
.\twitter_tada_mvl.csv
['Never once found a driver after several uses. What is going on???', '30 mins later still cannot find driver', 'Driver made me wait for 5min after accepting and canceled without warning. Tried to search for 15min and couldnt find any other driver. If this is the quality of drivers in the app, you can just forget about it.', "Change your login interface. I can't login because there is only a sign up option. And I can't sign up again cuz my mobile number is already used.", 'Fair is always super high as compare to Ryde and Grab. Planning to uninstall it soon.', 'Gps location inaccurate show far away for driver pickup', "I can't get my gps to get registered in and I can't lock the place where I want to go to. it's quite confusing please edit the appð\x9f\x91\x8d and I have a feeling that this TADA will give a tough fight to grab so get the app edited as soon as possible please", 'A

In [22]:
filename_sentiment_list[5:7]

['Gps location inaccurate show far away for driver pickup',
 "I can't get my gps to get registered in and I can't lock the place where I want to go to. it's quite confusing please edit the appð\x9f\x91\x8d and I have a feeling that this TADA will give a tough fight to grab so get the app edited as soon as possible please"]

### Cleaning

We will also perform cleaning

In [23]:
def clean_list_tokenise(reviews_list):
    # Tokenise the words
    tokenizer = RegexpTokenizer(r'\w+')
    return [tokenizer.tokenize(sentence) for sentence in reviews_list]

def clean_list_lemma(reviews_list):
    # Lemmatization
    lemma = WordNetLemmatizer()
    return [[lemma.lemmatize(word) for word in sentence] for sentence in reviews_list]

def clean_list_stopwords(reviews_list, stop_other=[]):
    def stopword_condition(word):
        word = word.lower()
        
        return word not in stopwords.words('english') \
            and word not in stop_other \
            and word[:5] != 'http' \
            and word[:5] != 'https' \
            and word[:2] != 'RT' \
            and word[0] != '@'
    
    return [[w for w in s if stopword_condition(w)] for s in reviews_list]

In [24]:
sentiment_filtered = clean_list_stopwords(clean_list_tokenise(filename_sentiment_list))
len(sentiment_filtered)

155

In [25]:
sentiment_filtered_sentences = [' '.join(s) for s in sentiment_filtered]
sentiment_filtered_sentences[:5]

['Never found driver several uses going',
 '30 mins later still cannot find driver',
 'Driver made wait 5min accepting canceled without warning Tried search 15min couldnt find driver quality drivers app forget',
 'Change login interface login sign option sign cuz mobile number already used',
 'Fair always super high compare Ryde Grab Planning uninstall soon']

### Segregating by Sentiment

We will segregate the reviews by sentiment. This is achieved by the `analyse_sentiment_vader` function, which uses vader to help us analyse the degree to which the particular review or comment is positive, negative or neutral. It also computes a compound score ranging from -1 to 1 which takes into account all the scores. More reading for the sentiment analysis are available at:

- https://opensourceforu.com/2016/12/analysing-sentiments-nltk/
- http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html

In [26]:
def analyse_sentiment_vader(df, col_name):
    sid = SentimentIntensityAnalyzer()
    vader = lambda text: sid.polarity_scores(text)
    
    df['vader'] = df[col_name].apply(vader)
    df = pd.merge(df, df['vader'].apply(pd.Series), left_index=True, right_index=True)
    return df.drop(['vader'], axis=1)

In [27]:
df_sentiment_filtered = pd.DataFrame(sentiment_filtered_sentences, columns=['comment'])
df_sentiment_filtered = analyse_sentiment_vader(df_sentiment_filtered, 'comment')
df_sentiment_filtered.head(10)

,comment,neg,neu,pos,compound
0,Never found driver several uses going,0.000,1.000,0.000,0.0000
1,30 mins later still cannot find driver,0.000,1.000,0.000,0.0000
2,Driver made wait 5min accepting canceled witho...,0.088,0.697,0.215,0.4090
3,Change login interface login sign option sign ...,0.000,0.894,0.106,0.0772
4,Fair always super high compare Ryde Grab Plann...,0.000,0.563,0.437,0.7351
5,Gps location inaccurate show far away driver p...,0.000,1.000,0.000,0.0000
6,get gps get registered lock place want go quit...,0.195,0.558,0.247,0.1689
7,Awesome ride hailing app much better grab,0.000,0.417,0.583,0.7906
8,TADA drivers tendencies turn pick another cust...,0.282,0.653,0.065,-0.6495
9,cant even sign n apps say already use,0.000,1.000,0.000,0.0000


In [28]:
df_sentiment_filtered.tail(10)

,comment,neg,neu,pos,compound
145,Cannot find place,0.000,1.000,0.000,0.0000
146,find destination location,0.000,1.000,0.000,0.0000
147,App crashes creating account Using s8,0.000,0.694,0.306,0.2960
148,publish article CNA app even opened,0.000,1.000,0.000,0.0000
149,App keeps crashing launch,0.000,1.000,0.000,0.0000
150,Bad app Cannot register,0.538,0.462,0.000,-0.5423
151,Good initiative,0.000,0.256,0.744,0.4404
152,Pls fix app auto close rate app properly,0.000,0.843,0.157,0.0772
153,Application keep crashing,0.000,1.000,0.000,0.0000
154,Unable open app,0.000,1.000,0.000,0.0000


In [29]:
pos_list = list(df_sentiment_filtered[df_sentiment_filtered['compound'] >= 0]['comment'])
neg_list = list(df_sentiment_filtered[df_sentiment_filtered['compound'] < 0]['comment'])

len(pos_list), len(neg_list)

(124, 31)

In [30]:
pos_list_clean = clean_list_lemma(clean_list_tokenise(pos_list))
neg_list_clean = clean_list_lemma(clean_list_tokenise(neg_list))

In [31]:
len(pos_list_clean), len(neg_list_clean)

(124, 31)

Ignore code below

### Topic Modelling

In [32]:
neg_dict = corpora.Dictionary(neg_list_clean)
doc_term_matrix = [neg_dict.doc2bow(doc) for doc in neg_list_clean]

print(doc_term_matrix[0:10])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1)], [(8, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(3, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)], [(3, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)], [(8, 1), (31, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)], [(4, 1), (8, 2), (34, 2), (35, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1)], [(8, 1), (50, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)], [(8, 1), (33, 1), (35, 1), (39, 1), (56, 2), (60, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1)], [(31, 1), (50, 1), (84, 1), (85, 1), (86, 1)], [(8, 1), (56, 1), (76, 1), (87, 1), (88, 2), 

In [44]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
#doc_term_matrix = frequency of terms of all documents
#dictionary = all unique terms
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word=neg_dict, passes=50)

In [45]:
print(ldamodel.print_topics(num_topics=3, num_words=5))

[(0, '0.039*"keep" + 0.039*"sign" + 0.039*"code" + 0.034*"App" + 0.034*"location"'), (1, '0.108*"app" + 0.040*"ride" + 0.037*"Grab" + 0.034*"try" + 0.029*"using"'), (2, '0.113*"driver" + 0.076*"app" + 0.056*"time" + 0.037*"price" + 0.033*"Grab"')]


In [46]:
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, neg_dict)

D:\Windows\88B90FF3D0D3454AAACE9B026E2829E3.TMP\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.117372 -0.070228       1        1  48.593254
0      0.149184 -0.033198       2        1  27.709854
1     -0.031812  0.103426       3        1  23.696892, topic_info=     Category       Freq          Term      Total  loglift  logprob
term                                                               
8     Default  26.000000        driver  26.000000  30.0000  30.0000
3     Default  29.000000           app  29.000000  29.0000  29.0000
56    Default  13.000000          time  13.000000  28.0000  28.0000
99    Default   5.000000          keep   5.000000  27.0000  27.0000
159   Default   5.000000          sign   5.000000  26.0000  26.0000
153   Default   5.000000          code   5.000000  25.0000  25.0000
146   Default   4.000000           try   4.000000  24.0000  24.0000
97    Default   4.000000           App   4.000000  23.0000  23.0000
218   Default   4.000000      location   4.000000  22.0000  22.0000
160   Default   4.000000  verification   4.000000  21.0000  21.0000
35    Default   9.000000          ride   9.000000  20.0000  20.0000
16    Default   3.000000         using   3.000000  19.0000  19.0000
39    Default   5.000000       service   5.000000  18.0000  18.0000
236   Default   3.000000          work   3.000000  17.0000  17.0000
201   Default   3.000000         crash   3.000000  16.0000  16.0000
91    Default   7.000000         still   7.000000  15.0000  15.0000
213   Default   5.000000        number   5.000000  14.0000  14.0000
92    Default   2.000000          sure   2.000000  13.0000  13.0000
30    Default   2.000000        Cannot   2.000000  12.0000  12.0000
173   Default   2.000000           new   2.000000  11.0000  11.0000
143   Default   9.000000         price   9.000000  10.0000  10.0000
94    Default   2.000000          used   2.000000   9.0000   9.0000
183   Default   2.000000     expensive   2.000000   8.0000   8.0000
88    Default   3.000000          apps   3.000000   7.0000   7.0000
154   Default   5.000000          good   5.000000   6.0000   6.0000
187   Default   1.000000       whether   1.000000   5.0000   5.0000
138   Default   1.000000            go   1.000000   4.0000   4.0000
54    Default   1.000000         place   1.000000   3.0000   3.0000
148   Default   1.000000          open   1.000000   2.0000   2.0000
2     Default   1.000000       another   1.000000   1.0000   1.0000
...       ...        ...           ...        ...      ...      ...
138    Topic3   1.410271            go   1.857207   1.1645  -4.3649
54     Topic3   1.410268         place   1.857204   1.1645  -4.3649
148    Topic3   1.410124          open   1.857222   1.1644  -4.3650
2      Topic3   1.408373       another   1.857627   1.1630  -4.3662
228    Topic3   0.805865     reinstall   1.252550   0.9988  -4.9245
223    Topic3   0.805865       problem   1.252550   0.9988  -4.9245
7      Topic3   0.805759      customer   1.252553   0.9987  -4.9246
215    Topic3   0.805759       Pricing   1.252553   0.9987  -4.9246
221    Topic3   0.805461      launched   1.252588   0.9983  -4.9250
145    Topic3   0.805425       similar   1.252595   0.9982  -4.9250
194    Topic3   0.803589        review   1.253021   0.9956  -4.9273
172    Topic3   0.783047       happens   1.257181   0.9664  -4.9532
195    Topic3   0.783047        simply   1.257181   0.9664  -4.9532
20     Topic3   0.771757    especially   1.252890   0.9553  -4.9677
88     Topic3   2.052040          apps   3.910202   0.7951  -3.9898
33     Topic3   1.237094          book   2.469468   0.7486  -4.4959
35     Topic3   4.419432          ride   9.253248   0.7009  -3.2226
134    Topic3   0.815649          easy   1.863590   0.6136  -4.9124
184    Topic3   0.859951         going   1.968287   0.6118  -4.8595
124    Topic3   0.806753          went   1.863066   0.6029  -4.9234
174    Topic3   0.806181      properly   1.863280   0.6020  -4.9241
141    Topic3  

# Topic Modelling (Positive)

In [36]:
pos_dict = corpora.Dictionary(pos_list_clean)
doc_term_matrix = [neg_dict.doc2bow(doc) for doc in pos_list_clean]

print(doc_term_matrix[0:10])

[[(8, 1), (184, 1)], [(8, 1), (53, 1), (91, 1), (202, 1)], [(3, 1), (8, 2), (17, 1), (161, 1), (202, 1), (210, 1)], [(94, 1), (159, 2), (213, 1), (220, 1)], [(44, 1), (133, 1)], [(8, 1), (175, 1), (218, 1)], [(1, 1), (3, 1), (54, 1), (76, 3), (137, 1), (138, 1)], [(3, 1), (35, 1), (103, 1)], [(34, 1), (88, 1), (89, 1), (147, 1), (159, 1), (235, 1)], [(3, 1)]]


In [42]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
#doc_term_matrix = frequency of terms of all documents
#dictionary = all unique terms
ldamodel = Lda(doc_term_matrix, num_topics=2, id2word=pos_dict, passes=50)

In [43]:
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, pos_dict)

D:\Windows\88B90FF3D0D3454AAACE9B026E2829E3.TMP\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.082753  0.0       1        1  80.466454
0     -0.082753  0.0       2        1  19.533545, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
99    Default   2.000000       Shall   2.000000  30.0000  30.0000
202   Default   3.000000        able   3.000000  29.0000  29.0000
3     Default  22.000000       going  22.000000  28.0000  28.0000
218   Default   2.000000       Fares   2.000000  27.0000  27.0000
39    Default   2.000000    Planning   2.000000  26.0000  26.0000
154   Default   3.000000       tried   3.000000  25.0000  25.0000
8     Default  18.000000        find  18.000000  24.0000  24.0000
173   Default   1.000000     replied   1.000000  23.0000  23.0000
22    Default   1.000000     quality   1.000000  22.0000  22.0000
33    Default   1.000000      number   1.000000  21.0000  21.0000
97    Default   3.000000   recommend   3.000000  20.0000  20.0000
44    Default  12.000000        soon  12.000000  19.0000  19.0000
11    Default   0.000000       still   0.000000  18.0000  18.0000
43    Default   0.000000        high   0.000000  17.0000  17.0000
96    Default   0.000000      notice   0.000000  16.0000  16.0000
53    Default   0.000000        show   0.000000  15.0000  15.0000
2     Default   0.000000       found   0.000000  14.0000  14.0000
56    Default   9.000000   confusing   9.000000  13.0000  13.0000
112   Default   1.000000        peak   1.000000  12.0000  12.0000
158   Default   0.000000      Damnit   0.000000  11.0000  11.0000
7     Default   0.000000      cannot   0.000000  10.0000  10.0000
215   Default   0.000000      though   0.000000   9.0000   9.0000
227   Default   0.000000        sure   0.000000   8.0000   8.0000
87    Default   0.000000       cheap   0.000000   7.0000   7.0000
194   Default   0.000000  reasonable   0.000000   6.0000   6.0000
20    Default   0.000000      forget   0.000000   5.0000   5.0000
143   Default   7.000000      15mins   7.000000   4.0000   4.0000
89    Default   1.000000      faster   1.000000   3.0000   3.0000
35    Default   7.000000        sign   7.000000   2.0000   2.0000
94    Default   1.000000   cancelled   1.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
275    Topic2   0.118890       quote   0.380937   0.4686  -6.6450
274    Topic2   0.118890      chance   0.380937   0.4686  -6.6450
273    Topic2   0.118890       bedok   0.380937   0.4686  -6.6450
97     Topic2   0.877588   recommend   3.136042   0.3595  -4.6460
89     Topic2   0.317516      faster   1.189959   0.3119  -5.6627
347    Topic2   0.118890       rival   0.380937   0.4686  -6.6450
268    Topic2   0.118890     account   0.380937   0.4686  -6.6450
269    Topic2   0.118890       allow   0.380937   0.4686  -6.6450
270    Topic2   0.118890        link   0.380937   0.4686  -6.6450
356    Topic2   0.118890     version   0.380937   0.4686  -6.6450
271    Topic2   0.118890          16   0.380937   0.4686  -6.6450
346    Topic2   0.118890        rite   0.380937   0.4686  -6.6450
267    Topic2   0.118890          FB   0.380937   0.4686  -6.6450
94     Topic2   0.339167   cancelled   1.687983   0.0282  -5.5967
279    Topic2   0.118890        card   0.380937   0.4686  -6.6450
134    Topic2   0.227913        Tada   1.297851  -0.1065  -5.9942
290    Topic2   0.118890      Actual   0.380937   0.4686  -6.6450
293    Topic2   0.118890    dropping   0.380937   0.4686  -6.6450
284    Topic2   0.118890      create   0.380937   0.4686  -6.6450
179    Topic2   0.156028    slightly   1.908503  -0.8710  -6.3732
292    Topic2   0.118890      double   0.380937   0.4686  -6.6450
285    Topic2   0.118890       hitch   0.380937   0.4686  -6.6450
291    Topic2   0.118890     arrival   0.380937   0.4686  -6.6450
175    Topic2   0.159093   Seriously   2.953002  -1.2881  -6.3537